## Tensorflow exploration ##
By Richard Sowers
* <r-sowers@illinois.edu>
* <https://publish.illinois.edu/r-sowers/>

Copyright 2019 University of Illinois Board of Trustees. All Rights Reserved.
Licensed under the MIT license

In [1]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation
import numpy
import scipy

from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)

Using TensorFlow backend.


Lets consider a 5-dimensional feature space, and let's make 1000 records.
Let's consider a binary classification problem, where the labels are either 0 or 1, and where the features are
[1,2,0,0,0]*label+noise

In [2]:
N_features=5
N_records=1000
labels = numpy.random.randint(2, size=(N_records, 1))
offset=numpy.array([1,2,0,0,0])
data=numpy.outer(labels,offset)+numpy.random.random((N_records, N_features))

In [3]:
labels[:10]

array([[1],
       [1],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0]])

In [4]:
data[:10]

array([[1.08748221, 2.22730974, 0.31437662, 0.17476588, 0.60709416],
       [1.41358642, 2.81635151, 0.1851304 , 0.70187653, 0.24035562],
       [0.57421909, 0.3489876 , 0.0569644 , 0.22881367, 0.66410256],
       [0.49725009, 0.51901598, 0.17472015, 0.57071585, 0.99675343],
       [1.81683511, 2.59437262, 0.97598907, 0.90156258, 0.59560793],
       [1.03242633, 2.0935771 , 0.06537172, 0.45173315, 0.37543483],
       [1.97535003, 2.16798329, 0.97278759, 0.76747487, 0.82423784],
       [1.63261582, 2.66873277, 0.47688233, 0.01313636, 0.35300609],
       [1.4920718 , 2.73009121, 0.46862834, 0.45740492, 0.13766274],
       [0.01088873, 0.75827826, 0.31995284, 0.98438345, 0.22023423]])

Let's try to construct a neural net of the form
$X_1=ReLU(M_0X_0+b_0) \qquad X_2=\sigma(M_1X_1+b_1)$ where
ReLU is as in <https://en.wikipedia.org/wiki/Rectifier_(neural_networks)> and $\sigma$ is as in <https://en.wikipedia.org/wiki/Sigmoid_function>.
Here:
* $X_0\in R^5$ is the feature vector
* $M_0\in R^{32\times 5}$, $b_0\in \R^{32}$, and ReLU is as in <https://en.wikipedia.org/wiki/Rectifier_(neural_networks)>; thus, pointwise, $X_1\in R^{32}$.
* $M_1\in R^{1\times 32}$, $b_1\in R^1$, and $\sigma$ is as in <https://en.wikipedia.org/wiki/Sigmoid_function>; thus $X_2\in \R^1$.

In [5]:
model = Sequential()
model.add(Dense(32, activation='relu', input_dim=5))
model.add(Dense(1, activation='sigmoid'))

Instructions for updating:
Colocations handled automatically by placer.


In [6]:
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [7]:
model.fit(data, labels, epochs=5, batch_size=32)

Instructions for updating:
Use tf.cast instead.
Epoch 1/5
1000/1000 [==============================] - 1s 686us/step - loss: 0.7875 - acc: 0.3800
Epoch 2/5
1000/1000 [==============================] - 0s 44us/step - loss: 0.5891 - acc: 0.8110
Epoch 3/5
1000/1000 [==============================] - 0s 69us/step - loss: 0.4643 - acc: 0.9000
Epoch 4/5
1000/1000 [==============================] - 0s 63us/step - loss: 0.3657 - acc: 0.9520
Epoch 5/5
1000/1000 [==============================] - 0s 81us/step - loss: 0.2842 - acc: 0.9680


Mathematically, this finds $M_0$, $b_0$, $M_1$ and $b_1$ such that the the loss between $X_2$ and the labels is minimized.  The prediction is $X_1$ itself.

In [8]:
model.predict(data[0:3])

array([[0.8164043],
       [0.9024216],
       [0.3227055]], dtype=float32)

Let's check that the weights are of the desired dimensions (matrix multiplication seems to be on the left)

In [9]:
for n,layer in enumerate(model.layers):
    weights = layer.get_weights()
    print("n={0:}".format(n))
    print("number of weight matrices:{0:}".format(len(weights)))
    for w in weights:
        print(w.shape)
    print("\n")

n=0
number of weight matrices:2
(5, 32)
(32,)


n=1
number of weight matrices:2
(32, 1)
(1,)




save the model

In [10]:
model.save("TF_test.hd5")

In [11]:
new_model = keras.models.load_model('TF_test.hd5')

In [12]:
new_model.predict(data[0:3])

array([[0.8164043],
       [0.9024216],
       [0.3227055]], dtype=float32)

continue training; https://stackoverflow.com/questions/42666046/loading-a-trained-keras-model-and-continue-training

In [13]:
new_model.fit(data, labels, epochs=5, batch_size=32)

Epoch 1/5
1000/1000 [==============================] - 1s 613us/step - loss: 0.2212 - acc: 0.9800
Epoch 2/5
1000/1000 [==============================] - 0s 48us/step - loss: 0.1702 - acc: 0.9860
Epoch 3/5
1000/1000 [==============================] - 0s 42us/step - loss: 0.1320 - acc: 0.9860
Epoch 4/5
1000/1000 [==============================] - 0s 49us/step - loss: 0.1040 - acc: 0.9880
Epoch 5/5
1000/1000 [==============================] - 0s 51us/step - loss: 0.0832 - acc: 0.9910
